In [ ]:
import JupyterFormatter
JupyterFormatter.enable_autoformat();

In [ ]:
function copy(file::String, dst::String)
    mkpath(joinpath(dst, dirname(file)))
    cp(file, joinpath(dst, file))
end

function build(app::String, basedir::String)
    name = split(app, "/")[end]
    run(`docker build -q -t rsys/$name $basedir/$name`)
end;

In [ ]:
function fetch_media_lists(basedir::String)
    app = "$basedir/fetch_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/fetch_media_lists", app)
    copy("notebooks/API/API", app)
    copy("environment/animeplanet", app)
    copy("environment/kitsu", app)
    copy("environment/mal", app)
    copy("environment/proxies", app)
    build(app, basedir)
end;

In [ ]:
function compress_media_lists(basedir::String)
    app = "$basedir/compress_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/compress_media_lists", app)
    copy("notebooks/ImportDatasets", app)
    copy("notebooks/ProcessData", app)
    copy("data/processed_data", app)
    build(app, basedir)
end;

In [ ]:
function bagofwords_jl(basedir::String)
    app = "$basedir/bagofwords_jl"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/bagofwords_jl", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("notebooks/TrainingAlphas/Baseline/BaselineHelper.ipynb", "$app/App/src")
    for x in ALL_MEDIUMS
        copy("data/alphas/$x/Baseline/rating/params.jld2", "$app/App/src")
    end
    copy("data/processed_data", "$app/App/src")
    build(app, basedir)
end;

In [ ]:
function bagofwords_py(basedir::String)
    for medium in ALL_MEDIUMS
        for metric in ALL_METRICS
            app = "$basedir/bagofwords_py_$(medium)_$(metric)"
            if ispath(app)
                rm(app; recursive = true)
            end
            cp("notebooks/Microservices/bagofwords_py", app)
            copy("notebooks/TrainingAlphas/BagOfWords/bagofwords.py", app)
            copy("data/alphas/$(medium)/BagOfWords/v1/$(metric)/model.pt", app)
            copy("data/alphas/$(medium)/BagOfWords/v1/$(metric)/config.json", app)
            build(app, basedir)
        end
    end
end;

In [ ]:
function nondirectional(basedir::String)
    app = "$basedir/nondirectional"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/nondirectional", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("data/processed_data", "$app/App/src")
    for m in ALL_MEDIUMS
        copy("environment/alphas/$m/Nondirectional", "$app/App/src")
    end
    build(app, basedir)
end;

In [ ]:
function transformer_jl(basedir::String)
    app = "$basedir/transformer_jl"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/transformer_jl", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("notebooks/TrainingAlphas/Transformer/Data.ipynb", "$app/App/src")
    copy("data/alphas/all/Transformer/v1/0/config.json", "$app/App/src")
    copy("data/processed_data", "$app/App/src")
    build(app, basedir)
end;

In [ ]:
function transformer_py(basedir::String)
    for medium in ALL_MEDIUMS
        app = "$basedir/transformer_py_$(medium)"
        if ispath(app)
            rm(app; recursive = true)
        end
        cp("notebooks/Microservices/transformer_py", app)
        copy("notebooks/TrainingAlphas/Transformer/transformer.py", app)
        copy("data/alphas/$(medium)/Transformer/v1/model.pt", app)
        copy("data/alphas/all/Transformer/v1/0/config.json", app)
        build(app, basedir)
    end
end;

In [ ]:
cd("../..")
basedir = "notebooks/Microservices/Images"
ALL_MEDIUMS = ["manga", "anime"]
ALL_METRICS = ["rating", "watch", "plantowatch", "drop"];

In [ ]:
if ispath(basedir)
    rm(basedir; recursive = true)
end
mkpath(basedir);

In [ ]:
fetch_media_lists(basedir);

In [ ]:
compress_media_lists(basedir);

In [ ]:
bagofwords_jl(basedir);

In [ ]:
bagofwords_py(basedir);

In [ ]:
nondirectional(basedir);

In [ ]:
transformer_jl(basedir);

In [ ]:
transformer_py(basedir);